In [1]:
from biosimulator_processes.compare import ComparisonDocument   
from process_bigraph import pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess not available. Error: 
PLEASE NOTE: Smoldyn is not correctly installed on your system which prevents you from using the SmoldynProcess. Please refer to the README for further information on installing Smoldyn.
TelluriumProcess registered successfully.


In [2]:
simulators = ['copasi', 'tellurium']  # , 'amici']
duration = 30
n_steps = 200
model_filepath = '../biosimulator_processes/model_files/sbml/Caravagna2010.xml'


comparison = ComparisonDocument(simulators, duration, n_steps, model_filepath)

In [3]:
pp(comparison.composite)

{ 'emitter': { '_type': 'step',
               'address': 'local:ram-emitter',
               'config': { 'emit': { 'floating_species_concentrations': 'tree[float]',
                                     'time': 'float'}},
               'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                           'time': ['time_store']}},
  'processes': { 'copasi_0': { '_type': 'process',
                               'address': 'local:copasi',
                               'config': { 'model': { 'model_source': '../biosimulator_processes/model_files/sbml/Caravagna2010.xml'}},
                               'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                                           'model_parameters': [ 'model_parameters_store'],
                                           'reactions': ['reactions_store'],
                                           'time': ['time_store']},
               

### ODE Simulator Comparison

In [4]:
from biosimulator_processes.compare import ComparisonDocument

sbml_model_path = '../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml'
simulators = ['copasi', 'copasi']
duration = 30 
n_steps = 50 
target_parameter = {
    'name': 'plasmin',
    'value': 2.01
}

ode_comparison_document = ComparisonDocument(
    simulators=simulators, 
    duration=duration, 
    num_steps=n_steps,
    model_filepath=sbml_model_path
)

In [5]:
pp(ode_comparison_document.composite)

{ 'emitter': { '_type': 'step',
               'address': 'local:ram-emitter',
               'config': { 'emit': { 'floating_species_concentrations': 'tree[float]',
                                     'time': 'float'}},
               'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                           'time': ['time_store']}},
  'processes': { 'copasi_0': { '_type': 'process',
                               'address': 'local:copasi',
                               'config': { 'model': { 'model_source': '../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml'}},
                               'inputs': { 'floating_species_concentrations': [ 'floating_species_concentrations_store'],
                                           'model_parameters': [ 'model_parameters_store'],
                                           'reactions': ['reactions_store'],
                                           'time': ['time_store']},
         